In [ ]:
import pickle
import pandas as pd
from collections import defaultdict

In [ ]:
uniprot = pd.read_csv("../uniprot.tsv.gz", sep='\t').set_index("Entry")[['Protein families', "Cross-reference (Pfam)"]]
prots = pd.read_pickle("results/prepare_proteins/protein_data_whole_label_none.pkl")
prots.index = prots.index.to_series().apply(lambda x:x.split("-")[1])
prots = prots.join(uniprot)

In [ ]:
prots = prots[prots['Cross-reference (Pfam)'].notna()].reset_index().drop_duplicates("index").set_index("index")
prots.drop("Protein families", axis=1, inplace=True)

In [ ]:
prots['fam'] = prots['Cross-reference (Pfam)'].apply(lambda x:x.split(";")[0] + ";")

In [ ]:
prots = prots[prots['fam'].isin(["PF13853;", "PF00076;"])]

In [ ]:
prots

In [ ]:
# prots['fam'] = prots['Cross-reference (Pfam)']
prots['nnodes'] = prots.data.apply(lambda x: x['x'].size(0))
prots = prots[prots['nnodes'].between(150, 1000)]

/home/ise20/miniconda3/envs/rindti/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
prots

In [ ]:
def add_fam(row):
    data = row['data']
    if "index_mapping" in data:
        del data['index_mapping']
    data['fam'] = row['fam']
    return data

prots['data'] = prots.apply(add_fam, axis=1)

In [ ]:
prots[['data']].to_pickle("/scratch/SCRATCH_NVME/ilya/pretrain_data/pfam_subset.pkl")

In [ ]:
from rindti.utils.data import PreTrainDataset

In [ ]:
ds = PreTrainDataset('/scratch/SCRATCH_NVME/ilya/pretrain_data/pfam_1000.pkl')

In [ ]:
ds.get_pfams()